In [119]:
import numpy as np
from math import factorial
from cmath import exp


def simplex_content(j, n_dims, signed, *args):
    """
    Compute simplex content (j-dim-volume) using Cayley-Menger Determinant
    :param j: dimension of simplex
    :param n_dims: dimension of R^n space
    :param signed: bool denoting whether to calculate signed content or unsigned content
    :param args: v0, v1, ... vectors for the coordinates of the vertices defining the simplex

    :return: vol: volume of the simplex
    """
    assert(n_dims == len(args[0]))
    assert(isinstance(signed, bool))
    n_vert = len(args)
    assert(n_vert == j+1)

    if n_dims > j:
        assert(not signed)

    if not signed:
        # construct Cayley-Menger matrix
        B = np.zeros([j+2, j+2])
        B[:, 0] = 1
        B[0, :] = 1
        B[0, 0] = 0
        for r in range(1, j+2):
            for c in range(r+1, j+2):
                vr = args[r-1]
                vc = args[c-1]
                B[r, c] = np.linalg.norm(vr-vc) ** 2
                B[c, r] = B[r, c]
        vol2 = (-1)**(j+1) / (2**j) / (factorial(j)**2) * np.linalg.det(B)
        if vol2 < 0:
            print("Warning: zeroing small negative number {0}".format(vol2))
        vol = np.sqrt(max(0, vol2))
    else:
        # matrix determinant
        mat = np.zeros([j, j])
        for r in range(j):
            mat[:, r] = args[r] - args[-1]
        vol = np.linalg.det(mat) / factorial(j)

    return vol

# def simplex_ft_bw_cpu(V, E, res=(256, 256), j=2):
#     assert(n_dims == len(res))  # consistent spacial dimensionality
#     assert(E.shape[0] == D.shape[0])  # consistent vertex numbers
#     assert(mode in ['density', 'mass'])

#     # number of columns in E
#     ghost = E.shape[1] == j and n_dims == j
#     assert (E.shape[1] == j+1 or ghost)
#     if ghost:
#         V = np.append(V, np.zeros([1, n_dims]), axis=0)
#         E = np.append(E, V.shape[0] - 1 + np.zeros([E.shape[0], 1], dtype=np.int), axis=1)
#     n_elem = E.shape[0]
#     n_vert = V.shape[0]
#     n_channel = D.shape[1]

#     # frequency tensor
#     omega = fftfreqs(res)
#     omega[tuple([0] * n_dims)] += 1e-3 # will get rid of this

#     # normalize frequencies
#     for dim in range(n_dims):
#         omega[..., dim] *= 2 * np.pi / t[dim]
    
#     for ie in range(E.shape[0]):
#         verts = V[E[ie]]
#         sig_list = []
#         esig_list = []
#         for vert in verts:
#             sig = np.sum(V[vert] * omega, axis=-1)
#             esig = np.exp(-1j * sig)
#             sig_list.append(sig)
#             esig_list.append(esig)
#         for i in range(j+1):
            

In [232]:
def simplex_ft_device(p, uvw):
    j = p.shape[0]-1
    n_dims = len(uvw)
    Fi = complex(0, 0)
    for dim in range(j+1):
        xi = p[dim]
        denom = 1
        uvw_dot_xi = complex(0, 0)
        for dd in range(n_dims):
            uvw_dot_xi += uvw[dd] * xi[dd]
        for d in range(j+1):
            if d != dim:
                xj = p[d]
                uvw_dot_xj = complex(0, 0)
                for dd in range(n_dims):
                    uvw_dot_xj += uvw[dd] * xj[dd]
                denom *= uvw_dot_xi - uvw_dot_xj
        Fi += exp(-1j*(uvw_dot_xi)) / denom
    return Fi

def pt_ft(V, E, k, j=2):
    k = k.copy() * 2 * np.pi
    F = complex(0,0)
    j = 2
    for ie in range(E.shape[0]):
        gamma = factorial(2) * simplex_content(2, 2, False, V[E[ie, 0]], V[E[ie, 1]], V[E[ie, 2]])
        fac = complex(0,0)
        if not k[0] == k[1] == 0:
            for dim in range(j+1):
                other_dims = [d for d in range(j+1) if d != dim]
                denom = 1
                sig_i = V[E[ie, dim]].dot(k)
                for d in other_dims:
                    sig_j = V[E[ie, d]].dot(k)
                    denom *= sig_i - sig_j
                fac += np.exp(-1j*sig_i) / denom
        else:
            fac = -0.5
        F += fac * gamma * (-1)
    return F

def diff(V, E, k, pt, delta):
    V_p = V.copy(); V_p[pt] += delta
    V_m = V.copy(); V_m[pt] -= delta
    F_p = pt_ft(V_p, E, k)
    F_m = pt_ft(V_m, E, k)
    dFdV = (F_p - F_m) / 2 / delta
    return dFdV

def pt_ft_bw(V, E, k, pt):
#     k = k.copy() * 2 * np.pi
    dFdV = np.zeros(2, dtype=np.complex_)
    j = 2
    # only loop over elems containing the point
    for ie in range(E.shape[0]):
        if pt[0] in E[ie]:
            seq = np.array([pt[0], (pt[0]+1) % 3, (pt[0]+2) % 3])
            if not k[0] == k[1] == 0:
                gamma = simplex_content(2, 2, False, V[E[ie, 0]], V[E[ie, 1]], V[E[ie, 2]])
                sig0 = V[E[ie, seq[0]]].dot(k)
                sig1 = V[E[ie, seq[1]]].dot(k)
                sig2 = V[E[ie, seq[2]]].dot(k)

                esig0 = np.exp(-1j*sig0)
                esig1 = np.exp(-1j*sig1)
                esig2 = np.exp(-1j*sig2)
                s01 = 1/(sig0 - sig1)
                s12 = 1/(sig1 - sig2)
                s20 = 1/(sig2 - sig0)
                S = -(esig0*s01*s20+esig1*s01*s12+esig2*s12*s20)
                fac = -esig0*s01*s20*(-1j-s01+s20)+esig1*s12*s01**2-esig2*s12*s20**2
                fac *= gamma
            else:
                fac = 0
                S = 0.5
            pt1 = V[E[ie, seq[1]]]
            pt2 = V[E[ie, seq[2]]]
            vec23 = np.array([pt1[1]-pt2[1], -pt1[0]+pt2[0]])
            dFdV += fac * k - S * vec23
                
    return dFdV[pt[1]]
            

In [235]:
from time import time
import numpy as np

# V = np.array([[0.2,0.2],
#               [0.8,0.5],
#               [0.8,0.8],
#               [0.6,0.8]])
# V = V+1e-10*np.random.rand(*V.shape)
E = np.array([[0,1,2],
              [2,3,0]])
V = np.array([[0.39,0.23],
              [0.91312,0.55],
              [0.93,0.81],
              [0.61,0.69]])
V = V+1e-10*np.random.rand(*V.shape)
D = np.ones((E.shape[0], 1))
# E = np.array([[0,1],
#               [1,2],
#               [2,3],
#               [3,0]])

In [238]:
k = np.array([0.,0.])
for d in deltas:
    print(d, diff(V, E, k, (0,0), d))
diff(V, E, k, (1,0), 1e-8)
pt_ft_bw(V, E, k, (1,0))

0.01 (-0.07000000004291351+0j)
0.001 (-0.07000000004288298+0j)
0.0001 (-0.07000000004225848+0j)
1e-05 (-0.07000000004225848+0j)
1e-06 (-0.07000000000478845+0j)
1e-07 (-0.07000000079582236+0j)
1e-08 (-0.06999999330181694+0j)


(-0.289999999992942+0j)

In [192]:
from matplotlib import pyplot as plt
deltas = np.array([10**(-i) for i in range(2,9)])
diffs = np.array([diff(V, E, k, (1,1), d) for d in deltas])
# plt.figure()
# plt.semilogx(deltas[1:], np.absolute(diffs[1:]-diffs[:-1]))
# plt.show()
print(diffs)
print(deltas)

[-1.86689827e-33+3.65918573e-33j -6.78037377e-36-4.20029814e-35j
 -1.15149340e-37-2.82482824e-37j -7.30830858e-40+6.92292686e-40j
  9.04763403e-42-1.01729678e-41j  4.85132541e-44+1.09723645e-45j
 -2.07521616e-46-1.28039187e-46j]
[1.e-02 1.e-03 1.e-04 1.e-05 1.e-06 1.e-07 1.e-08]


In [62]:
from transform import fftfreqs
freqs = fftfreqs((256,256))
print(freqs[187,61])

[-69.  61.]
